In [1]:
import numpy as np
import math
import cmath
import matplotlib.pyplot as plt
pi = math.pi

#Prints out numbers without "np.flat64" displaying
np.set_printoptions(legacy='1.25')

In [21]:
#Calculates B and X in a lumped element matching network.
def L_matching_network(Zo,ZL) :
    ZL_norm = ZL/Zo
    Zo_norm = 1
    ref = (ZL_norm-Zo_norm)/(ZL_norm+Zo_norm)
    print(ref)
    ref_mag = abs(ref)
    RL = ZL.real
    XL = ZL.imag
    if RL<Zo.real :
        print(f'Γ = {round(ref_mag,3)}, Outside 1 + jx circle')
        Xp = np.sqrt(RL*(Zo-RL)) - XL
        Xm = -np.sqrt(RL*(Zo-RL)) - XL
        Bp = np.sqrt((Zo-RL)/RL)/Zo
        Bm = -np.sqrt((Zo-RL)/RL)/Zo
        return [[Xp,Bp],[Xm,Bm]]
    if RL>Zo.real :
        print(f'Γ = {round(ref_mag,3)}, Inside 1 + jx circle')
        Bp = (XL + np.sqrt(RL/Zo)*np.sqrt(RL**2 + XL**2 - Zo*RL))/(RL**2 + XL**2)
        Bm = (XL - np.sqrt(RL/Zo)*np.sqrt(RL**2 + XL**2 - Zo*RL))/(RL**2 + XL**2)
        Xp = (1/Bp) + (XL*Zo)/RL - Zo/(Bp*RL)
        Xm = (1/Bm) + (XL*Zo)/RL - Zo/(Bm*RL)
        return [[Xp,Bp],[Xm,Bm]]
    else :
        print(f'ERROR: RL = Zo')
        return 0

In [22]:
ZL = complex(120,20)
Zo = 40

ans = L_matching_network(ZL, Zo)
print(ans)

(-0.5076923076923077-0.061538461538461535j)
Γ = 0.511, Outside 1 + jx circle
[[(57.00212496255788+7.017282255051754j), (0.01179155513616213-0.0005033253862245729j)], [(-57.00212496255788-7.017282255051754j), (-0.01179155513616213+0.0005033253862245729j)]]


In [2]:
#Calculates the lengths of stub for two lowest possible single stub in series with load and line.
def calc_single_stub_series(ZL, Zo, type) :
    ''' Inputs:
            ZL: load impedence (must be in complex form!)
            Zo: line impedence to match load to
            type: 'short' or 'open' for load type at end of stub.
        Returns: array of form [[d1,l1],[d2,l2]]
    '''

    Yo = 1/Zo
    YL = 1/ZL
    GL = YL.real
    BL = YL.imag

    if ((type != 'short') & (type != 'open')) : #check if type is valid.
        type1 = 'short'
        type2 = 'open'
        print(f'ERROR: incorrect type. Please choose {type1} or {type2}')
        return -1

    #follows algorithm from 5.2 in D. Pozar - Microwave Engineering, 4th ed (2012).
    #has to possible solutions, t_pos and t_neg, which correspond to a '+' or a '-'
    #   in the t equations.
    if GL != Yo :
        t_pos = (BL + np.sqrt((GL*(((Yo-GL)**2)+ BL**2))/Yo))/(GL-Yo)
        t_neg = (BL - np.sqrt((GL*(((Yo-GL)**2)+ BL**2))/Yo))/(GL-Yo)
    else :
        print('GL = Yo')
        t_pos = -BL/(2*Yo)
        t_neg = 0
    if t_pos >=0 : d_pos = (1/(2*pi))*np.arctan(t_pos)
    else : d_pos = (1/(2*pi))*(pi + np.arctan(t_pos))
    if t_neg >=0 : d_neg = (1/(2*pi))*np.arctan(t_neg)
    else : d_neg = (1/(2*pi))*(pi + np.arctan(t_neg))

    X_pos = ((GL**2)*t_pos - ((Yo-t_pos*BL)*(BL+t_pos*Yo))) / (Yo*(GL**2 + (BL + Yo*t_pos)**2))
    X_neg = ((GL**2)*t_neg - ((Yo-t_neg*BL)*(BL+t_neg*Yo))) / (Yo*(GL**2 + (BL + Yo*t_neg)**2))

    if type == 'short' :
        l_pos = (-1/(2*pi))*np.arctan(X_pos/Zo)
        l_neg = (-1/(2*pi))*np.arctan(X_neg/Zo)
    if type == 'open' :
        l_pos = (1/(2*pi))*np.arctan(Zo/X_pos)
        l_neg = (1/(2*pi))*np.arctan(Zo/X_neg)

    if l_pos < 0 :
        l_pos = l_pos + 0.5
    if l_neg < 0 :
        l_neg = l_neg + 0.5

    return [[d_pos,l_pos],[d_neg,l_neg]]


In [7]:
ZL = complex(3,0.5)
Zo = 1

ans = calc_single_stub_series(ZL, Zo, 'open')
print(ans)

[[0.4273179297519947, 0.11121084466349099], [0.09187992335640723, 0.38878915533650904]]


In [12]:
#Calculates the lengths of stub for two lowest possible single stub shunted with between load and line.
def calc_single_stub_shunt(ZL, Zo, type) :
    ''' Inputs:
            ZL: load impedence (must be in complex form!)
            Zo: line impedence to match load to
            type: 'short' or 'open' for load type at end of stub.
        Returns: array of form [[d1,l1],[d2,l2]]
    '''
    
    Yo = 1/Zo
    RL = ZL.real
    XL = ZL.imag

    if ((type != 'short') & (type != 'open')) : #check if type is valid.
        type1 = 'short'
        type2 = 'open'
        print(f'ERROR: incorrect type. Please choose {type1} or {type2}')
        return -1

    #follows algorithm from 5.2 in D. Pozar - Microwave Engineering, 4th ed (2012).
    #has to possible solutions, t_pos and t_neg, which correspond to a '+' or a '-'
    #   in the t equations.
    if RL != Zo :
        t_pos = (XL + np.sqrt((RL*(((Zo-RL)**2)+ XL**2))/Zo))/(RL-Zo)
        t_neg = (XL - np.sqrt((RL*(((Zo-RL)**2)+ XL**2))/Zo))/(RL-Zo)
    else :
        print('RL = Zo')
        t_pos = -XL/(2*Zo)
        t_neg = 0
    if t_pos >=0 : d_pos = (1/(2*pi))*np.arctan(t_pos)
    else : d_pos = (1/(2*pi))*(pi + np.arctan(t_pos))
    if t_neg >=0 : d_neg = (1/(2*pi))*np.arctan(t_neg)
    else : d_neg = (1/(2*pi))*(pi + np.arctan(t_neg))

    B_pos = ((RL**2)*t_pos - ((Zo-XL*t_pos)*(XL+Zo*t_pos))) / (Zo*(RL**2 + (XL + Zo*t_pos)**2))
    B_neg = ((RL**2)*t_neg - ((Zo-XL*t_neg)*(XL+Zo*t_neg))) / (Zo*(RL**2 + (XL + Zo*t_neg)**2))

    if type == 'open' :
        l_pos = (-1/(2*pi))*np.arctan(B_pos/Yo)
        l_neg = (-1/(2*pi))*np.arctan(B_neg/Yo)
    if type == 'short' :
        l_pos = (1/(2*pi))*np.arctan(Yo/B_pos)
        l_neg = (1/(2*pi))*np.arctan(Yo/B_neg)

    if l_pos < 0 :
        l_pos = l_pos + 0.5
    if l_neg < 0 :
        l_neg = l_neg + 0.5

    return [[d_pos,l_pos],[d_neg,l_neg]]


In [16]:
ZL = complex(3,0.5)
Zo = 1

ans = calc_single_stub_shunt(ZL, Zo, 'short')
print(ans)

[[0.17731792975199465, 0.11121084466349099], [0.3418799233564072, 0.388789155336509]]
